PCA
$$
a_i = argmin_{\| a \| = 1} \sum_{i=1}^n (x_i - a^T x_i a)^2
$$
It is sensitive to ouliers, because of sum of squares. Why not to try to find new components by minimizing not sum of squares with a help of another distance...?


We are going to do the same things as in PCA, but with different approaches in computing means and new components.

![alt text](https://i.ibb.co/0hmqr1f/image.png)


Left: A zero-mean dataset represented as a set of points.
Center: The same data represented as a set of one-dimensional
subspaces. Right: The blue dotted subspace is the average.



Note that here we are speaking about feature-outliers. A simple example is burnt pixel in an image.
![alt text](https://cs8.pikabu.ru/post_img/big/2016/04/27/6/1461749915137084224.jpg)

**Definition 1** The Grassman Manifold $Gr(k,D)$ is the space of all $k$-dimensional subspaces of $\mathbb{R}^D$

In fact, we will need only $Gr(1, D)$. Which is isomorphic to the following set:
$$
[u] = \{u, -u \}, ~ \| u \| = 1.
$$
This manifold also can be written as the quotient 
$$
S^{D-1} / \{ \pm 1 \}.
$$
See "J. Lee. Introduction to Smooth Manifolds. Springer, 2002. 3" for details.

![alt text](https://i.ibb.co/Wv4ggjW/image.png)


From now we live on the $Gr(1,D)$. Thus, we need a kind of a squared distance here. Particularly, for the computation of new components.
$$
dist^2_{S^{D-1}}(u_1, u_2) = \frac{1}{2} \| u_1 - u_2 \|^2 = 1 - u_1^T u_2.
$$
Weighted average for this distance is
$$
q = \frac{\mu(\omega_{1:N}, u_{1:N})}{\| \mu(\omega_{1:N}, u_{1:N}) \|},
$$
here $u_n = \frac{x_n}{\| x_n \|}$

Moreover, we can choose another mean function $\mu_{rob}(\omega, u)$, such that it will possess desired robustness properties. For example, we can use trimmed mean.


TrimmedMean$([a_1, a_2, a_3, a_4, a_5], 20\%) = \frac{1}{3} \sum_{i=2}^4 a_i$

TrimmedMean$([a_1, a_2, a_3, a_4, a_5], 50\%) =$ Median$([a_1, a_2, a_3, a_4, a_5]) = a_3$  

In [0]:
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.utils import as_float_array, check_array, check_random_state
from sklearn.utils.validation import check_is_fitted



Now we are going to implement a class that is computing principal component is the robust way, using Trimmed Mean.

In practice, it is more convenient to wipe out according to the $k$th smallest and $k$th largest:
TrimmedMean$([a_1, a_2, a_3, a_4, a_5], 1) = \frac{1}{3} \sum_{i=2}^4 a_i$

TrimmedMean$([a_1, a_2, a_3, a_4, a_5], 2) = a_3$  

In [0]:
def _trimmed_mean_1d(arr, k):
    """Calculate trimmed mean on a 1d array.

    Trim values largest than the k'th largest value or smaller than the k'th
    smallest value

    Parameters
    ----------
    arr: ndarray, shape (n,)
        The one-dimensional input array to perform trimmed mean on

    k: int
        The thresholding order for trimmed mean

    Returns
    -------
    trimmed_mean: float
        The trimmed mean calculated
    """
    kth_smallest = np.partition(arr, k)[k-1]
    kth_largest = -np.partition(-arr, k)[k-1]

    ########################
    #code here!
    ########################

In [0]:
### test your might
test_array = np.array([1, 2, 3, 7, 5])
ans = _trimmed_mean_1d(test_array, 0)
assert ans == 5.0, "Something is wrong..."

Now, we can simply apply this for the whole dataset

In [0]:


def _trimmed_mean(X, trim_proportion):
    """Calculate trimmed mean on each column of input matrix

    Parameters
    ----------
    X: ndarray, shape (n_samples, n_features)
        The input matrix to perform trimmed mean on

    trim_proportion: float
        The proportion of trim. Largest and smallest 'trim_proportion' are
        trimmed when calculating the mean.

    Returns
    -------
    trimmed_mean: ndarray, shape (n_features,)
        The trimmed mean calculated on each column
    """
    n_samples, n_features = X.shape
    n_trim = int(n_samples * trim_proportion)
    return np.apply_along_axis(_trimmed_mean_1d, 0, X, k=n_trim)


Also, we will need supplementary function for reorthogonalization of the components

In [0]:


def _reorth(basis, target, rows=None, alpha=0.5):
    """Reorthogonalize a vector using iterated Gram-Schmidt

    Parameters
    ----------
    basis: ndarray, shape (n_features, n_basis)
        The matrix whose rows are a set of basis to reorthogonalize against

    target: ndarray, shape (n_features,)
        The target vector to be reorthogonalized

    rows: {array-like, None}, default None
        Indices of rows from basis to use. Use all if None

    alpha: float, default 0.5
        Parameter for determining whether to do a second reorthogonalization.

    Returns
    -------
    reorthed_target: ndarray, shape (n_features,)
        The reorthogonalized vector
    """
    if rows is not None:
        basis = basis[rows]
    norm_target = np.linalg.norm(target)

    norm_target_old = 0
    n_reorth = 0

    while norm_target < alpha * norm_target_old or n_reorth == 0:
        for row in basis:
            t = np.dot(row, target)
            target = target - t * row

        norm_target_old = norm_target
        norm_target = np.linalg.norm(target)
        n_reorth += 1

        if n_reorth > 4:
            # target in span(basis) => accpet target = 0
            target = np.zeros(basis.shape[0])
            break

    return target


Now there is a big job below: code that `TGA` class. It has a structure based on `PCA` from `sklearn`, so there is a lot of methods but we will pay attention only on the key one which is `._fit`


The key part of this method is computation of the next TGA component. It is to be computed in accordance to the following formula:
$$
\omega_n \leftarrow sign(u_n^T q_{i-1}); ~ q_i \leftarrow \frac{\mu_{rob}(\omega_{1:N}, u_{1:N})}{\| \mu_{rob}(\omega_{1:N}, u_{1:N}) \|},
$$
here $u_n = \frac{x_n}{\| x_n \|}$.

Note that components are being computed based on the previous one.

There is another strategy for finding principal 

Below we will implement the most valuable of the `._fit` method that is computation of the next component. The other part are technical routine.

In [1]:
def get_new_component(k, prev_components, X, n_features, n_samples, trim_proportion, tol):
  if k == 0:
    # initialize using a few EM iterations
    mu = rng.rand(n_features) - 0.5
    mu = mu / np.linalg.norm(mu)
    for i in range(3):
      dots = np.dot(X, mu)
      mu = np.dot(dots.T, X)
      mu = mu / np.linalg.norm(mu)

  # grassmann average
  for i in range(n_samples):
    prev_mu = prev_components[k-1]
    ########################
    #code the formula above
    ########################
    if np.max(np.abs(mu - prev_mu)) < tol:
      break

  #possibly re-orthonormalize
  if k > 0:
      mu = _reorth(prev_components[:k-1], mu)
      mu = mu / np.linalg.norm(mu)
  return mu

In [0]:
# test your might
X_tst  = np.array([[1, 2, 3, 7], [112, 113 , 114, 123], [10, 22, -123, 57]])
prev_components_tst = [1, 1, 2]
n_samples_tst, n_features_tst = X_tst.shape
ans_tst = get_new_component(1, prev_components_tst, X_tst, n_features_tst, n_samples_tst, 0.1, 1e-5)
assert (np.linalg.norm(ans_tst - np.array([0.16152159, 0.35534751, 0., 0.92067308])) < 1e-8), "Something is wrong..."

I have gathered all the code in one file for you, now let us go after the experiments!